In [3]:
print ('Hello world') # Начали также, как начинали все великие прогаммисты

Hello world


# Шаг 1: выбор темы
Мы - активные пользователи маркетплейсов, заметили, что цены на товары меняются в течении времени. Также, мы любим читать новости, где узнали, что Федеральная антимонопольная служба РФ (далее ФАС) активно следит за деятельностью маркетплейсов. Наша команда решила тоже поисследовать рынок маркетплейсов, а именно продавцов на "www.wildberries.ru" (далее ВБ). 

В ходе нашего проекта мы хотим проверить следующую гипотезу: 
##### Цена на один и тот же товар изменяется у различных продавцов синхронно, что может свидетельстовать об использовании продавцами алгоритмических роботов ценообразования или олигополистического сговора на рынке. Выберем некоторые категории товаров. Затем соберем ряды цен на один и тот же товар у разных продавцов и протестируем их на наличие синхронных структурных сдвигов

# Шаг 2: сбор данных
Мы решили выделить 6 категорий товаров, собрать данные по 5 товарам внутри категории (30 товаров) и собрать данные по ценам на каждый товар у 5 разных продавцов (итого 150 товаров). Однако мы забыли добавить стиральные машины, поэтому товаров не 150, а 145. В таблице "Цены ВБ.xlsx" столбцами являются артикулы товаров, выше написано к какому типу товара относиться артикул, а еще выше его категория. 

### Теперь немного про то, почему мы выбирали именно эти категории:

#### 1) Смартфоны и электроника
- Примеры: Смартфон iPhone 13, Повербанк, XBOX, Ноутбук, Экшн-камера

#### 2) Бытовая техника
- Примеры: Телевизор, Микроволновка, Утюг, Холодильник
  
#### 3) Брендовая одежда и обувь
- Примеры: Джинсы, Черная футболка, Кросовки, Кофта черная, Трусы мужские
  
#### 4)  Косметика и парфюмерия
- Примеры: Крем для рук, Тушь, Карандаш для губ, Шампунь, Гель для бровей

#### 5) Спортивные товары
- Примеры: Очки, Волейбольный мяч, Теннисный мяч, Коврик спортивный, Гантели

#### 6) Товары для дома
- Примеры: Держатель бумажных полотенец, Ваза для цветов, Коврик придверный, Кактус комнатный, Свечи светодиодные 



### Все эти категории мы считаем подходящими под нашу гипотизу потому что они удовлетворяют следующим условиям: 
#### Высокая конкуренция и динамическое ценообразование: 
Во всех этих категориях товаров присутствует высокая конкуренция, и продавцы стремятся приспособить свои цены к изменениям у конкурентов, что делает использование алгоритмических роботов эффективным инструментом для подстройки цен.

#### Частые скидки и акции: 
Периодические распродажи и скидки создают возможности для сговора или манипулирования ценами с целью согласования уровней цен на товары.

#### Высокая стоимость товаров:
На рынке дорогих товаров (например, смартфонов, бытовой техники) всегда существует стимул для манипуляций с ценами, так как повышение или понижение цен значительно влияет на прибыль.

#### Схожесть товаров: 
Продавцы часто продают одинаковые товары, что упрощает использование роботов для синхронизации цен между ними или даже для сговора о ценах.



### Мы написали парсер, который каждый час собирает цены по артикулам, код представлен в файле с названием main_parser.py. Мы используем api ВБ для парсинга. Подробные коментарии по коду парсера можно прочитать внутри файла. 

# Шаг 3: предварительная обработка

In [8]:
# Импортируем нужные библиотеки
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
# Считаываем данные из собранного файла
df = pd.read_excel('Цены ВБ.xlsx', skiprows=2)
df.head()

,Дата,277503519,278324976,248946359,306096104,255796969,182404057,90116547,153088675,153203825,...,209240844,216291819,156970391,209242389,234136855,9391510,18364130,309985228,15396161,126668875
0,2025-02-17 14:00:00,47510,45999,47510,53153,51298,6259,7705,7586,7468,...,696,693,611,726,251,412,412,341,720,561
1,2025-02-17 15:00:00,47510,45999,47510,53153,51298,6259,7705,7586,7468,...,696,693,611,726,251,412,412,341,720,561
2,2025-02-17 16:00:00,47510,45999,47510,53153,51298,6259,7705,7586,7468,...,688,685,604,718,251,401,401,332,712,555
3,2025-02-17 17:00:00,47510,45999,47510,53153,51298,6259,7705,7586,7468,...,688,685,604,718,251,401,401,332,712,555
4,2025-02-17 18:00:00,47510,45999,47510,53153,51298,6259,7705,7586,7468,...,688,685,604,718,251,401,401,332,712,555


В чем заключается предварительная обработка?
Мы: 
1. Удалим столбец с пропусками (столбец с очками), так как пропусков сильно больше чем данных. В процессе парсинга мы заметили, что товар больше недоступен и заменили его на аналогичный. 
2. Заполним пропущенные данные (это 1 столбец с другими очками, который служит заменой) следующим образом: Возьмем первую доступную цену и заполним пропущенные столбцы выше, это не должно сильно повлиять на наш анализ и выводы, так как мы заполним меньше 20 сттрок таким образом и всего на 1 товар. Это необходимо из-за того, что мы заменяли один товар.
3. Сделаем столбец с датой индексом
4. Поделим даты по категориям. Мы знаем что в каждой категории 5 товаров и в каждом товаре по 5 продавцов. Есть 1 исключение, это бытовая техника, там 4 товара по 5 продавцов, так как мы забыли добавить стиральную машину.



In [11]:
df["Дата"] = pd.to_datetime(df["Дата"])
df.set_index("Дата", inplace=True)

In [13]:
df.head()

,277503519,278324976,248946359,306096104,255796969,182404057,90116547,153088675,153203825,270872197,...,209240844,216291819,156970391,209242389,234136855,9391510,18364130,309985228,15396161,126668875
Дата,,,,,,,,,,,,,,,,,,,,,
2025-02-17 14:00:00,47510,45999,47510,53153,51298,6259,7705,7586,7468,10417,...,696,693,611,726,251,412,412,341,720,561
2025-02-17 15:00:00,47510,45999,47510,53153,51298,6259,7705,7586,7468,10417,...,696,693,611,726,251,412,412,341,720,561
2025-02-17 16:00:00,47510,45999,47510,53153,51298,6259,7705,7586,7468,10417,...,688,685,604,718,251,401,401,332,712,555
2025-02-17 17:00:00,47510,45999,47510,53153,51298,6259,7705,7586,7468,10417,...,688,685,604,718,251,401,401,332,712,555
2025-02-17 18:00:00,47510,45999,47510,53153,51298,6259,7705,7586,7468,10417,...,688,685,604,718,251,401,401,332,712,555


In [14]:
df.drop(columns=[171289236], inplace=True) # Удаляем столбец данные по которому не получилось собрать

In [15]:
df.loc[df[249820881] == 'Замена товара 171289236', 249820881] = 3198 # Заменяем 20 строк на цену первого доступного периода

In [16]:
# Делим на категории
df_cat = pd.read_excel('Цены ВБ.xlsx')
df_cat = df_cat.loc[:,~(df_cat==171289236).any()]
col =[]
categories = list(df_cat.columns)
categories = list(filter(lambda x: 'Unnamed' not in x, categories))
categories=categories[1::]

for i in range(6):
    if i == 0:
        col+=[categories[i]]*20
    else:
        col+=[categories[i]]*25
catagories_colums = col

In [19]:
# Делим на товары
df_goods = pd.read_excel('Цены ВБ.xlsx', skiprows = 1)
df_goods = df_goods.loc[:,~(df_goods==171289236).any()]
good =[]
goods = list(df_goods.columns)
goods = list(filter(lambda x: 'Unnamed' not in x, goods))
goods=goods[1::]
for i in range(29):
    good+=[goods[i]]*5
goods_columns = good

In [23]:
# Получили таблицу, в которой артикулы соотносятся с товарами и категориями
data={'Категория':catagories_colums,
      'Товар':goods_columns,
      'Артикул': list(df.columns)

}
df_catalog = pd.DataFrame(data)

df_catalog


,Категория,Товар,Артикул
0,Бытовая техника,Телевизор,277503519
1,Бытовая техника,Телевизор,278324976
2,Бытовая техника,Телевизор,248946359
3,Бытовая техника,Телевизор,306096104
4,Бытовая техника,Телевизор,255796969
...,...,...,...
140,Товары для дома,Свечи светодиодные,9391510
141,Товары для дома,Свечи светодиодные,18364130
142,Товары для дома,Свечи светодиодные,309985228
143,Товары для дома,Свечи светодиодные,15396161
